In [13]:
import yaml
import os, sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
import random
import pickle

In [7]:
sys.path.append("/data/rsg/nlp/sdobers/ruslan/diffdock-protein/src")

from data import load_data, get_data
from data.utils import DIPSLoader

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [01:13<00:00,  2.72it/s]
/data/rsg/nlp/sdobers/ruslan/diffdock-protein/src/geom_utils/torus.py:46: RuntimeWarning: invalid value encountered in divide
  score_ = grad(x, sigma[:, None], N=100) / p_


In [8]:
class Dict2Class:
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])
            
with open("/data/rsg/nlp/sdobers/ruslan/diffdock-protein/config/dips_esm_batchwise_loading.yaml", "r") as f:
    args_loaded = yaml.safe_load(f)

args_dict = {} 
for small_dict in args_loaded.values():
    args_dict = {**args_dict, **small_dict}
    
args_dict = {
    **args_dict, 
    "samples_directory": "/data/rsg/nlp/sdobers/DIPS/confidence_full_20_poses",
    "debug": False,
    "recache": False,
    "use_randomized_confidence_data": False,
    "mode": ""
}

args = Dict2Class(args_dict)

fold = 0

In [18]:
n_batches = DIPSLoader.get_n_batches()["test"]
batch_indexes = list(range(n_batches))
test_batches = [load_data(args, split="test", batch=index, verbose=False)  for index in tqdm(batch_indexes)]
test_batches_loaders = [get_data(batch, fold, args, for_reverse_diffusion=True)["test"] for batch in test_batches]

In [11]:
def write_pdb(item, graph, part, path):
    lines = to_pdb_lines(item, graph, part)
    with open(path, "w") as file:
        file.writelines(lines)


def to_pdb_lines(visualization_values, graph, part):
    assert part in ("ligand", "receptor", "both"), "Part should be ligand or receptor"
    parts = ["ligand", "receptor"] if part == "both" else [part]

    lines = []
    for part in parts:
        this_vis_values = visualization_values[part]
        this_vis_values = {k: v.strip() if type(v) is str else v for k, v in this_vis_values.items()}
        for i, resname in enumerate(this_vis_values["resname"]):
            xyz = graph[part].pos[i]

            line = f'ATOM  {i + 1:>5} {this_vis_values["atom_name"][i]:>4} '
            line = line + f'{resname} {this_vis_values["chain"][i]}{this_vis_values["residue"][i]:>4}    '
            line = line + f'{xyz[0]:>8.3f}{xyz[1]:>8.3f}{xyz[2]:>8.3f}'
            line = line + '  1.00  0.00          '
            line = line + f'{this_vis_values["element"][i]:>2} 0\n'
            lines.append(line)

    return lines

In [52]:
PATH_dips_40s_large = '/data/rsg/nlp/sdobers/amine/diffdock-protein/storage/largest_model_140_epochs/dips_test_40s_lowtemp.pkl'
with open(PATH_dips_40s_large, "rb") as f:
    results = pickle.load(f)

In [53]:
len(results)

100

In [54]:
gt = [res[0][0] for res in results]

all_pred = [[graph_confidence[0] for graph_confidence in res[1:41]] for res in results]

In [55]:
len(all_pred)

100

In [56]:
directory = "/data/rsg/nlp/sdobers/ruslan/diffdock-protein/visualization_results"

visualization_items = []

for gt_graph in gt:
    for batch in test_batches_loaders:
        if gt_graph.name in batch.pdb_ids:
            visualization = batch.get_visualization_values(pdb_name=gt_graph.name)
            visualization_items.append(visualization)

In [57]:
import os

for gt_graph, item in zip(gt, visualization_items):
    name = gt_graph.name.replace('/', '_').replace('.dill', '')
    current_dir = f"{directory}/{name}"
    os.makedirs(current_dir, exist_ok=True)
    write_pdb(item, gt_graph, "receptor", f"{current_dir}/receptor.pdb")
    write_pdb(item, gt_graph, "ligand", f"{current_dir}/ligand_gt.pdb")

In [58]:
for iteration_i in range(40):
    for graph_i in range(len(all_pred)):
        graph = all_pred[graph_i][iteration_i]
        item = visualization_items[graph_i]
        
        name = graph.name.replace('/', '_').replace('.dill', '')
        
        current_dir = f"{directory}/{name}"
        write_pdb(item, graph, "ligand", f"{current_dir}/ligand_{iteration_i}.pdb")